In [1]:
# Import libraries
## Basic libs
import pandas as pd
import numpy as np
import warnings
## Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Configure libraries
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10, 10)
plt.style.use('seaborn')

In [2]:
# Load dataset
df_bank_ready = pd.read_csv('https://raw.githubusercontent.com/M0MKD/C-M_Dataset/main/OurTraffic.csv')


print('Shape of dataframe:', df_bank_ready.shape)
# Removing spaces in the columns names
df_bank_ready.columns = [i.strip() for i in df_bank_ready.columns ]
df_bank_ready.head()

Shape of dataframe: (562, 84)


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.174.128-192.168.174.130-6522-49202-6,192.168.174.130,49202,192.168.174.128,6522,6,25/05/2022 07:07:28 PM,9070260,3,6,...,0,70476.0,0.0,70476.0,70476.0,5969094.0,0.000,5969094.0,5969094.0,Malicious
1,192.168.174.128-192.168.174.130-6522-49202-6,192.168.174.130,49202,192.168.174.128,6522,6,25/05/2022 07:07:37 PM,1473,1,1,...,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,Malicious
2,192.168.174.128-192.168.174.130-6522-49203-6,192.168.174.130,49203,192.168.174.128,6522,6,25/05/2022 07:08:09 PM,9038119,3,6,...,0,47350.0,0.0,47350.0,47350.0,5984062.0,0.000,5984062.0,5984062.0,Malicious
3,192.168.174.128-192.168.174.130-6522-49203-6,192.168.174.130,49203,192.168.174.128,6522,6,25/05/2022 07:08:18 PM,3137,1,1,...,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,Malicious
4,192.168.174.128-192.168.174.130-6522-49201-6,192.168.174.128,6522,192.168.174.130,49201,6,25/05/2022 07:06:48 PM,105608331,66,42,...,0,9698229.0,22100000.0,54700000.0,47159.0,7895486.5,3042827.195,13700000.0,5051273.0,Malicious


In [3]:
print(df_bank_ready['Label'].value_counts())

Malicious    544
Benign        18
Name: Label, dtype: int64


In [4]:
from sklearn import impute
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan,
                        strategy ='mean')
 
data = (df_bank_ready)
# Fitting the data to the imputer object
imputer = imputer.fit(data)
 
# Imputing the data    
data = imputer.transform(data)
 
print("Imputed Data : \n", data)
print(df_bank_ready.isnull().sum())

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: '192.168.174.128-192.168.174.130-6522-49202-6'

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_cols = ['Src Port', 'Dst Port', 'Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts']
df_bank_ready[num_cols] = scaler.fit_transform(df_bank_ready[num_cols])

print(df_bank_ready.head())

                                        Flow ID           Src IP  Src Port  \
0  192.168.174.128-192.168.174.130-6522-49202-6  192.168.174.130 -0.030254   
1  192.168.174.128-192.168.174.130-6522-49202-6  192.168.174.130 -0.030254   
2  192.168.174.128-192.168.174.130-6522-49203-6  192.168.174.130 -0.030196   
3  192.168.174.128-192.168.174.130-6522-49203-6  192.168.174.130 -0.030196   
4  192.168.174.128-192.168.174.130-6522-49201-6  192.168.174.128 -2.496823   

            Dst IP  Dst Port  Protocol               Timestamp  Flow Duration  \
0  192.168.174.128  0.626634 -1.529112  25/05/2022 07:07:28 PM      -0.057033   
1  192.168.174.128  0.626634 -1.529112  25/05/2022 07:07:37 PM      -0.475585   
2  192.168.174.128  0.626634 -1.529112  25/05/2022 07:08:09 PM      -0.058517   
3  192.168.174.128  0.626634 -1.529112  25/05/2022 07:08:18 PM      -0.475509   
4  192.168.174.130  7.447105 -1.529112  25/05/2022 07:06:48 PM       4.398495   

   Tot Fwd Pkts  Tot Bwd Pkts  ...  Fwd Seg 

In [6]:
df_bank_ready.drop(['Flow ID', 'Src IP', 'Dst IP', 'Timestamp','Idle Min'],axis = 1,inplace = True)

In [7]:
df_bank_ready['Label'] = df_bank_ready['Label'].apply(lambda x: x.upper())
df_bank_ready.Label.value_counts()

MALICIOUS    544
BENIGN        18
Name: Label, dtype: int64

In [8]:
from sklearn.preprocessing import OneHotEncoder

df_bank_ready['Label'] = df_bank_ready['Label'].apply(lambda x: x.upper())


# Encode target value
encoded = {'BENIGN':0,'MALICIOUS':1,  }
df_bank_ready['Label'] = df_bank_ready['Label'].map(encoded)

print('Shape of dataframe:', df_bank_ready.shape)
(df_bank_ready.head())

Shape of dataframe: (562, 79)


,Src Port,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Label
0,-0.030254,0.626634,-1.529112,-0.057033,-0.060406,0.318155,41,2,39,0,...,2,0,70476.0,0.0,70476.0,70476.0,5969094.0,0.000,5969094.0,1.0
1,-0.030254,0.626634,-1.529112,-0.475585,-0.108187,-0.203441,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,1.0
2,-0.030196,0.626634,-1.529112,-0.058517,-0.060406,0.318155,41,2,39,0,...,2,0,47350.0,0.0,47350.0,47350.0,5984062.0,0.000,5984062.0,1.0
3,-0.030196,0.626634,-1.529112,-0.475509,-0.108187,-0.203441,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,1.0
4,-2.496823,7.447105,-1.529112,4.398495,1.444688,4.073645,7608,28899,1460,0,...,33,0,9698229.0,22100000.0,54700000.0,47159.0,7895486.5,3042827.195,13700000.0,1.0


In [28]:
feature = df_bank_ready.drop('Label', axis=1)


target = df_bank_ready['Label']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature , target, 
                                                    shuffle = True, 
                                                    test_size=0.2, 
                                                    random_state=1)

print('Shape of training feature:', X_train.shape)
print('Shape of testing feature:', X_test.shape)
print('Shape of training Label:', y_train.shape)
print('Shape of training Label:', y_test.shape)

Shape of training feature: (449, 78)
Shape of testing feature: (113, 78)
Shape of training Label: (449,)
Shape of training Label: (113,)


In [10]:
def evaluate_model(model, x_test, y_test):
    from sklearn import metrics

    # Predict Test Data 
    y_pred = model.predict(x_test)

    # Calculate accuracy, precision, recall, f1-score, and kappa score
    acc = metrics.accuracy_score(y_test, y_pred)
    prec = metrics.precision_score(y_test, y_pred)
    rec = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)
    kappa = metrics.cohen_kappa_score(y_test, y_pred)

    # Calculate area under curve (AUC)
    y_pred_proba = model.predict_proba(x_test)[::,1]
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
    auc = metrics.roc_auc_score(y_test, y_pred_proba)

    # Display confussion matrix
    cm =metrics.confusion_matrix(y_test, y_pred)

    return {'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1, 'kappa': kappa, 
            'fpr': fpr, 'tpr': tpr, 'auc': auc, 'cm': cm}

In [20]:
y_train

378    1.0
395    1.0
474    1.0
19     1.0
289    1.0
      ... 
129    1.0
144    1.0
72     1.0
235    1.0
37     1.0
Name: Label, Length: 449, dtype: float64

In [30]:
X_train = np.nan_to_num(X_train)
y_train = np.nan_to_num(y_train)


In [31]:
X_train.shape,y_train.shape

((449, 78), (449,))

In [32]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)


RandomForestClassifier(random_state=0)

In [ ]:
# Evaluate Model
rf_eval = evaluate_model(rf, X_test, y_test)

# Print result
print('Accuracy:', rf_eval['acc'])
print('Precision:', rf_eval['prec'])
print('Recall:', rf_eval['rec'])
print('F1 Score:', rf_eval['f1'])
print('Cohens Kappa Score:', rf_eval['kappa'])
print('Area Under Curve:', rf_eval['auc'])
print('Confusion Matrix:\n', rf_eval['cm'])

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [50, 80, 100],
    'max_features': [2, 3, 4],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 300, 500]
}

# Create a base model
rf_grids = RandomForestClassifier(random_state=0)

# Initiate the grid search model
grid_search = GridSearchCV(estimator=rf_grids, param_grid=param_grid, scoring='recall',
                           cv=5, n_jobs=-1, verbose=2)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

ValueError: ignored

In [ ]:
best_grid = grid_search.best_estimator_

# Evaluate Model
best_grid_eval = evaluate_model(best_grid, X_test, y_test)

# Print result
print('Accuracy:', best_grid_eval['acc'])
print('Precision:', best_grid_eval['prec'])
print('Recall:', best_grid_eval['rec'])
print('F1 Score:', best_grid_eval['f1'])
print('Cohens Kappa Score:', best_grid_eval['kappa'])
print('Area Under Curve:', best_grid_eval['auc'])
print('Confusion Matrix:\n', best_grid_eval['cm'])

AttributeError: ignored

In [ ]:
df_bank_ready['deposit_prediction'] = rf.predict(feature)
df_bank_ready['deposit_prediction'] = df_bank_ready['deposit_prediction'].apply(lambda x: 'yes' if x==0 else 'no')

# Save new dataframe into csv file
df_bank_ready.to_csv('deposit_prediction.csv', index=False)

print(df_bank_ready.head(10))

AttributeError: ignored

In [ ]:
from joblib import dump, load

# Saving model
dump(rf, 'bank_deposit_classification.joblib')
# Loading model
# clf = load('bank_deposit_classification.joblib')

['bank_deposit_classification.joblib']